Implementing an Auotoencoder network for anomaly detection based on the proposed procedure in following paper:
Collision Detection for Robot Manipulators Using Unsupervised Anomaly Detection Algorithms Kyu Min Park , Younghyo Park , Sangwoong Yoon , and Frank C. Park , Fellow, IEEE



In [1]:
import pandas as pd

Now reload data from data set 

In [6]:
data_df = pd.read_csv("Data sets/dataCircularPath_f15N.csv")
data_df.head()

,ZeitInSec,ZeitInNanoSec,axisTauExtMsr_LBR_iiwa_14_R820_1[0],axisTauExtMsr_LBR_iiwa_14_R820_1[1],axisTauExtMsr_LBR_iiwa_14_R820_1[2],axisTauExtMsr_LBR_iiwa_14_R820_1[3],axisTauExtMsr_LBR_iiwa_14_R820_1[4],axisTauExtMsr_LBR_iiwa_14_R820_1[5],axisTauExtMsr_LBR_iiwa_14_R820_1[6],cartForce1_X,...,axisQMsr_LBR_iiwa_14_R820_1[4],axisQMsr_LBR_iiwa_14_R820_1[5],axisQMsr_LBR_iiwa_14_R820_1[6],CartPosMsr1_X,CartPosMsr1_Y,CartPosMsr1_Z,CartPosCmd1_X,CartPosCmd1_Y,CartPosCmd1_Z,CollisionType
0,1683998916,952000000,-0.125497,0.708675,-0.337407,-0.036705,0.088480,-0.358344,0.252109,-0.198948,...,-3.550534,-47.931647,-87.098549,0.057388,700.068897,60.028683,-0.002835,700.062685,60.081810,NoColl
1,1683998916,954000000,-0.124175,0.709554,-0.337068,-0.036897,0.088484,-0.358388,0.252119,-0.200875,...,-3.550534,-47.931661,-87.098549,0.057364,700.068860,60.028681,-0.002835,700.062685,60.081810,NoColl
2,1683998916,956000000,-0.129727,0.704701,-0.338558,-0.035497,0.088108,-0.358359,0.252142,-0.192838,...,-3.550540,-47.931647,-87.098539,0.057515,700.068878,60.028637,-0.002835,700.062685,60.081810,NoColl
3,1683998916,958000000,-0.133682,0.708605,-0.340409,-0.035942,0.087970,-0.358350,0.252133,-0.187162,...,-3.550540,-47.931654,-87.098532,0.057514,700.068828,60.028644,-0.002835,700.062685,60.081810,NoColl
4,1683998916,960000000,-0.123046,0.703267,-0.338948,-0.031795,0.088011,-0.358534,0.252085,-0.202570,...,-3.550534,-47.931654,-87.098525,0.057417,700.068777,60.028608,-0.001535,700.062711,60.076895,NoColl


Sampling signals into 60ms window with an Interval of 4ms.
remember, that the kuka sampling rate set to 500Hz. So I took the mean of each 2 rows. But first, pick the non-collision data and remove the last column 

In [7]:
data_df_nocoll =  data_df[data_df['CollisionType'].str.contains('NoColl')]
data_df_nocoll = data_df_nocoll.drop(["CollisionType","ZeitInSec","ZeitInNanoSec"],axis=1) #dropping cols with string or integer value
data_df_nocoll = data_df_nocoll.astype('float16')
data_df_nocoll.shape


(180000, 32)

In [8]:

col_names = data_df_nocoll.columns
data_df_nocoll = pd.DataFrame(data_df_nocoll.values.reshape(-1,2,data_df_nocoll.shape[1]).mean(1),columns=data_df_nocoll.columns) 
data_df_nocoll.columns = col_names
data_df_nocoll.head()
#data_df_nocoll.columns


,axisTauExtMsr_LBR_iiwa_14_R820_1[0],axisTauExtMsr_LBR_iiwa_14_R820_1[1],axisTauExtMsr_LBR_iiwa_14_R820_1[2],axisTauExtMsr_LBR_iiwa_14_R820_1[3],axisTauExtMsr_LBR_iiwa_14_R820_1[4],axisTauExtMsr_LBR_iiwa_14_R820_1[5],axisTauExtMsr_LBR_iiwa_14_R820_1[6],cartForce1_X,cartForceVar1_X,cartForce1_Y,...,axisQMsr_LBR_iiwa_14_R820_1[3],axisQMsr_LBR_iiwa_14_R820_1[4],axisQMsr_LBR_iiwa_14_R820_1[5],axisQMsr_LBR_iiwa_14_R820_1[6],CartPosMsr1_X,CartPosMsr1_Y,CartPosMsr1_Z,CartPosCmd1_X,CartPosCmd1_Y,CartPosCmd1_Z
0,-0.124817,0.708984,-0.337402,-0.036804,0.088501,-0.358398,0.252197,-0.199951,-1.501953,-0.351318,...,58.5625,-3.550781,-47.9375,-87.125,0.057373,700.0,60.03125,-0.002834,700.0,60.09375
1,-0.131714,0.706543,-0.339355,-0.035706,0.088013,-0.358398,0.252197,-0.189941,-1.501953,-0.352051,...,58.5625,-3.550781,-47.9375,-87.125,0.057526,700.0,60.03125,-0.002834,700.0,60.09375
2,-0.123413,0.706055,-0.338867,-0.033569,0.087891,-0.358398,0.251953,-0.201904,-1.501953,-0.360352,...,58.5625,-3.550781,-47.9375,-87.125,0.057434,700.0,60.03125,0.002220,700.0,60.06250
3,-0.149414,0.689453,-0.346436,-0.028931,0.086670,-0.358398,0.251709,-0.163818,-1.501953,-0.350830,...,58.5625,-3.550781,-47.9375,-87.125,0.058472,700.0,60.03125,0.039856,700.0,59.93750
4,-0.302734,0.544922,-0.402832,0.027054,0.080872,-0.361816,0.251465,0.058105,-1.501953,-0.330078,...,58.5625,-3.550781,-47.9375,-87.125,0.065796,700.0,60.03125,0.135498,700.0,59.56250
